In [ ]:
import pandas as pd
import wandb
import numpy as np

In [3]:
api_wandb = wandb.Api()

run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/mnipfdm0').scan_history())

In [4]:
run

,_step,progressivity,vkdnw_lambda_1,vkdnw_lambda_5,vkdnw_lambda_8,vkdnw_dim,complexity,vkdnw_lambda_7,arch,trainability,...,vkdnw_lambda_0,vkdnw_chisquare,flops,num_layers,vkdnw_lambda_4,_timestamp,vkdnw_lambda_3,_runtime,model_size,vkdnw_lambda_2
0,0,-Infinity,0.000244,0.194629,7.421612,244,381031544.0,1.810366,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.190722,...,8.016209e-07,190.459459,381031544.0,14,0.068927,1.729607e+09,0.019113,17.287027,3627705.0,0.003603
1,1,-0.017285,0.000006,0.093808,4.455290,248,434595992.0,1.264581,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.186515,...,1.113927e-07,143.468354,434595992.0,14,0.021177,1.729607e+09,0.003287,18.087931,3688617.0,0.000377
2,2,-0.709576,0.000057,0.790009,55.465851,270,402400248.0,10.776753,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.185046,...,9.500461e-06,116.229508,402400248.0,15,0.140649,1.729607e+09,0.024333,18.874209,3652009.0,0.001123
3,3,-0.338296,0.000059,0.523543,13.819300,274,410379800.0,4.165202,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-0.175802,...,2.507468e-06,157.880597,410379800.0,15,0.115631,1.729607e+09,0.014269,19.651100,3679761.0,0.000829
4,4,-0.247345,0.000061,0.276663,11.580867,280,417704712.0,2.290114,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.239618,...,1.729702e-07,176.583333,417704712.0,14,0.097906,1.729607e+09,0.023244,20.503012,3675097.0,0.004763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76246,76246,-0.789663,0.000065,2.150551,226.252106,320,226964764.0,63.217213,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",-0.377860,...,7.038095e-07,120.836364,226964764.0,14,0.445777,1.729693e+09,0.117521,86387.795209,1126625.0,0.009473
76247,76247,-Infinity,0.000005,1.528132,59.838116,274,388668292.0,16.284239,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.268079,...,1.852322e-07,113.655172,388668292.0,13,0.023683,1.729693e+09,0.000678,86388.898207,3500993.0,0.000048
76248,76248,-0.329206,0.000002,2.289304,69.226440,320,396934984.0,13.941657,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.341497,...,1.763321e-07,120.461538,396934984.0,14,0.731657,1.729693e+09,0.137673,86390.667794,3063665.0,0.006313
76249,76249,-Infinity,0.000006,1.378431,40.040604,274,438466012.0,10.738747,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.236537,...,9.194015e-07,147.724138,438466012.0,13,0.333114,1.729693e+09,0.069614,86391.824915,3775769.0,0.002764


In [8]:
for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')

In [9]:
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

In [10]:
run.shape

(76251, 28)

In [18]:
run['vkdnw_rank'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True)

In [36]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('az_nas', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
19012,19012.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K5(32,...",449037468.0,322.0,15.0,3685273.0,44.805483,41972.0,21.869541,44.239056,44.209899,44.321068


In [34]:
run.loc[run['vkdnw_dim']==346, ['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
31828,31828.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",449079804.0,346.0,15.0,2313505.0,43.579347,76177.0,22.465845,43.912776,43.73765,44.455743


In [42]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
64775,64775.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",448750524.0,326.0,15.0,3782881.0,44.752932,66546.0,22.283814,44.661257,44.687104,44.652615


In [44]:
#run.loc[run['arch']=='SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,40,2,24,1)SuperResIDWE1K5(40,64,2,48,4)SuperResIDWE1K5(64,192,2,144,5)SuperResIDWE6K5(192,176,2,176,3)SuperConvK1BNRELU(176,2048,1,1)', ]

# Compare runs

In [ ]:
import pandas as pd
import wandb
import numpy as np

In [ ]:
run_small = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/mnipfdm0').scan_history())
run_large = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/73qqqhqt').scan_history())